In [40]:
import datetime
import random
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.backends import cudnn
import torch.nn as nn
from torcheval.metrics.functional import multiclass_accuracy

import segmentation_models_pytorch as smp
import segmentation_models_pytorch.utils

import cv2
from keras.utils import to_categorical

import PIL.Image

from os.path import join as pjoin
import os

from tqdm import tqdm
import json

In [41]:
from utils import test_i_sample, test_x_sample
from my_dataset import MyDataset
from tf_callback import images_to_probs, plot_classes_preds, plot_confusion_matrix, plot_to_image
from metrics import runningScore

In [42]:
model_name = f'model_{input()}'

In [43]:
print(model_name)

model_0307_0435


In [44]:
params_dict = None

In [45]:
with open(pjoin('models', model_name, 'params.json'), 'r') as f:
    params_dict = json.load(f)

In [46]:
num_classes = 6

In [47]:
# python_seed = 245
# np_seed = 123
# torch_seed = 321
# torch_cuda_seed = 111
# learning_rate = 0.001
step = params_dict['step']
split_test = params_dict['split_test']
path_data = 'data'

In [48]:
im_shape=(401,701)
iline, xline = im_shape
i_locations = np.arange(0, iline, step)
x_locations = np.arange(0, xline, step)

In [49]:
test_i_mask = [i not in i_locations for i in range(0, iline)]
test_x_mask = [x not in x_locations for x in range(0, xline)]

In [50]:
seismic = np.load(pjoin(path_data,'train','train_seismic.npy'))
labels  = np.load(pjoin(path_data,'train','train_labels.npy' ))

In [51]:
test_seismic_ix = seismic[test_i_mask][:,test_x_mask]
test_labels_ix = labels[test_i_mask][:,test_x_mask]

In [52]:
test_data_shape = test_seismic_ix.shape

In [53]:
print(test_data_shape)

(396, 693, 255)


In [54]:
# Создаем U-Net модель с энкодером resnet34
model = smp.Unet(
    encoder_name="resnet18", # resnet18 
    # encoder_weights="swsl", # можно обучать с нуля
    in_channels=1,
    classes=6,  # Количество классов для сегментации
)

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [56]:
model = model.to(device)

In [57]:
model.load_state_dict(torch.load(pjoin('models', model_name, f'best_0_{model_name}.pth')))

<All keys matched successfully>

In [58]:
test_data_shape

(396, 693, 255)

In [19]:
pred_labels_cat_i = np.zeros((test_data_shape[0], test_data_shape[1], test_data_shape[2], 6))

In [20]:
for i in tqdm(range(test_data_shape[0])):
    im = test_seismic_ix[i]
    lbl = to_categorical(test_labels_ix[i], num_classes=6)
    _, lbl_cat, pred_lbl_cat = test_i_sample(model, im, lbl, False)
    pred_labels_cat_i[i] = pred_lbl_cat

100%|██████████| 396/396 [06:10<00:00,  1.07it/s]


In [21]:
np.save(pjoin('models', model_name, 'pred_labels_cat_i.npy'), pred_labels_cat_i)

In [22]:
pred_labels_cat_x = np.zeros((test_data_shape[0], test_data_shape[1], test_data_shape[2], 6))

In [23]:
for i in tqdm(range(test_data_shape[1])):
    im = test_seismic_ix[:,i]
    lbl = to_categorical(test_labels_ix[:,i], num_classes=6)
    _, lbl_cat, pred_lbl_cat = test_x_sample(model, im, lbl, False)
    pred_labels_cat_x[:,i,:,:] = pred_lbl_cat

100%|██████████| 693/693 [04:58<00:00,  2.33it/s]


In [24]:
np.save(pjoin('models', model_name, 'pred_labels_cat_x.npy'), pred_labels_cat_x)

In [59]:
loss = [smp.utils.losses.DiceLoss(), nn.CrossEntropyLoss()]
metrics = [smp.utils.metrics.IoU(threshold=0.5)]

In [26]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(pjoin('models', model_name, 'runs'))

In [60]:
for i in tqdm(range(test_data_shape[0])):
    lbl = torch.from_numpy(np.expand_dims(np.moveaxis(to_categorical(test_labels_ix[i],num_classes=6), -1, 0), axis=0))
    pred_i = torch.from_numpy(np.expand_dims(np.moveaxis(pred_labels_cat_i[i], -1, 0), axis=0))
    pred_x = torch.from_numpy(np.expand_dims(np.moveaxis(pred_labels_cat_x[i], -1, 0), axis=0))

    li = torch.round(loss[0](pred_i, lbl) + loss[1](pred_i, lbl), decimals=2)
    mi = torch.round(metrics[0](pred_i, lbl), decimals=2)
    lx = torch.round(loss[0](pred_x, lbl) + loss[1](pred_x, lbl), decimals=2)
    mx = torch.round(metrics[0](pred_x, lbl), decimals=2)

    writer.add_scalar('test/(i_algo) loss along i', li, i)
    writer.add_scalar('test/(i_algo) metric along i', mi, i)
    writer.add_scalar('test/(x_algo) loss along i', lx, i)
    writer.add_scalar('test/(x_algo) metric along i', mx, i)

100%|██████████| 396/396 [00:53<00:00,  7.46it/s]


In [61]:
for i in tqdm(range(test_data_shape[1])):
    lbl = torch.from_numpy(np.expand_dims(np.moveaxis(to_categorical(test_labels_ix[:,i],num_classes=6), -1, 0), axis=0))
    pred_i = torch.from_numpy(np.expand_dims(np.moveaxis(pred_labels_cat_i[:,i], -1, 0), axis=0))
    pred_x = torch.from_numpy(np.expand_dims(np.moveaxis(pred_labels_cat_x[:,i], -1, 0), axis=0))

    li = torch.round(loss[0](pred_i, lbl) + loss[1](pred_i, lbl), decimals=2)
    mi = torch.round(metrics[0](pred_i, lbl), decimals=2)
    lx = torch.round(loss[0](pred_x, lbl) + loss[1](pred_x, lbl), decimals=2)
    mx = torch.round(metrics[0](pred_x, lbl), decimals=2)

    writer.add_scalar('test/(i_algo) loss along x', li, i)
    writer.add_scalar('test/(i_algo) metric along x', mi, i)
    writer.add_scalar('test/(x_algo) loss along x', lx, i)
    writer.add_scalar('test/(x_algo) metric along x', mx, i)
    

100%|██████████| 693/693 [00:50<00:00, 13.69it/s]


In [62]:
lbl_i.shape

torch.Size([396, 6, 693, 255])

In [63]:
pred_labels_cat_i = np.load(pjoin('models', model_name, 'pred_labels_cat_i.npy'))
pred_labels_cat_x = np.load(pjoin('models', model_name, 'pred_labels_cat_x.npy'))

MemoryError: Unable to allocate 3.13 GiB for an array with shape (419874840,) and data type float64

In [ ]:
lbl_i = torch.from_numpy(np.moveaxis(to_categorical(test_labels_ix,num_classes=6), -1, 1))
pred_i = torch.from_numpy(np.moveaxis(pred_labels_cat_i, -1, 1))
lbl_x = torch.from_numpy(np.moveaxis(to_categorical(test_labels_ix,num_classes=6), (1, -1), (0, 1)))
pred_x = torch.from_numpy(np.moveaxis(pred_labels_cat_x, (1, -1), (0, 1)))

li = torch.round(loss[0](pred_i, lbl_i) + loss[1](pred_i, lbl_i), decimals=2)
mi = torch.round(metrics[0](pred_i, lbl_i), decimals=2)
lx = torch.round(loss[0](pred_x, lbl_x) + loss[1](pred_x, lbl_x), decimals=2)
mx = torch.round(metrics[0](pred_x, lbl_x), decimals=2)

print(f'i_algo: loss={li}, metric={mi}')
print(f'x_algo: loss={lx}, metric={mx}')

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3358998720 bytes.

In [ ]:
test_labels_ix.shape, pred_labels_cat_i.shape, pred_labels_cat_x.shape

((392, 686, 255), (392, 686, 255, 6), (392, 686, 255, 6))

In [ ]:
pred_labels_i = np.argmax(pred_labels_cat_i, axis=-1)
pred_labels_x = np.argmax(pred_labels_cat_x, axis=-1)

In [ ]:
pred_labels_i.shape, pred_labels_x.shape

((392, 686, 255), (392, 686, 255))

In [ ]:
running_metrics_test = runningScore(num_classes)

In [ ]:
metric_names = ['Pixel Acc', 'Mean Class Acc', 'Freq Weighted IoU', 'Mean IoU']

In [ ]:
running_metrics_test.update(test_labels_ix, pred_labels_i)
score, class_iu = running_metrics_test.get_scores()
running_metrics_test.reset()

test_res_dict_i = {}

for i, m in enumerate(metric_names):
    _s = score[f'{m}: ']
    print(f'{m}: \t{_s}')
    test_res_dict_i[f'{m}'] = _s

for i, _ca in enumerate(score['Class Accuracy: ']):
    print(f'Class[{i}] Accuracy:\t{_ca}')
    test_res_dict_i[f'Class[{i}] Accuracy'] = _ca

    print(f'Class[{i}] IoU:     \t{class_iu[i]}')
    test_res_dict_i[f'Class[{i}] IoU'] = class_iu[i]

Pixel Acc: 	0.973998024865923
Mean Class Acc: 	0.9070583471725234
Freq Weighted IoU: 	0.9504360344946858
Mean IoU: 	0.8637697212892076
Class[0] Accuracy:	0.992789743170252
Class[0] IoU:     	0.9801244318166096
Class[1] Accuracy:	0.9438319127143
Class[1] IoU:     	0.9022877849231831
Class[2] Accuracy:	0.9893851310728955
Class[2] IoU:     	0.9740207627042012
Class[3] Accuracy:	0.9386464823429194
Class[3] IoU:     	0.8858961846746283
Class[4] Accuracy:	0.904464824182109
Class[4] IoU:     	0.7901789602697624
Class[5] Accuracy:	0.6732319895526655
Class[5] IoU:     	0.6501102033468612


In [ ]:
running_metrics_test.update(test_labels_ix, pred_labels_x)
score, class_iu = running_metrics_test.get_scores()
running_metrics_test.reset()

test_res_dict_x = {}

for i, m in enumerate(metric_names):
    _s = score[f'{m}: ']
    print(f'{m}: \t{_s}')
    test_res_dict_x[f'{m}'] = _s

for i, _ca in enumerate(score['Class Accuracy: ']):
    print(f'Class[{i}] Accuracy:\t{_ca}')
    test_res_dict_x[f'Class[{i}] Accuracy'] = _ca

    print(f'Class[{i}] IoU:     \t{class_iu[i]}')
    test_res_dict_x[f'Class[{i}] IoU'] = class_iu[i]

Pixel Acc: 	0.9717394829651977
Mean Class Acc: 	0.8921466915087365
Freq Weighted IoU: 	0.9463613778218636
Mean IoU: 	0.8462412199406376
Class[0] Accuracy:	0.9937088089132197
Class[0] IoU:     	0.980925161754491
Class[1] Accuracy:	0.9460882224851264
Class[1] IoU:     	0.9025041178486072
Class[2] Accuracy:	0.9874127408144003
Class[2] IoU:     	0.9699971429445104
Class[3] Accuracy:	0.93390328320508
Class[3] IoU:     	0.8779610783324893
Class[4] Accuracy:	0.892359972137966
Class[4] IoU:     	0.7663971774347683
Class[5] Accuracy:	0.599407121496627
Class[5] IoU:     	0.5796626413289593


In [ ]:
with open(pjoin('models', model_name, 'test_res_dict_i.json'), 'w') as f:
    json.dump(test_res_dict_i, f)
with open(pjoin('models', model_name, 'test_res_dict_x.json'), 'w') as f:
    json.dump(test_res_dict_x, f)

In [ ]:
pred_labels_ix = np.argmax((pred_labels_cat_i+pred_labels_cat_x)*0.5, axis=-1)

In [ ]:
running_metrics_test.update(test_labels_ix, pred_labels_ix)
score, class_iu = running_metrics_test.get_scores()
running_metrics_test.reset()

test_res_dict_ix = {}

for i, m in enumerate(metric_names):
    _s = score[f'{m}: ']
    print(f'{m}: \t{_s}')
    test_res_dict_ix[f'{m}'] = _s

for i, _ca in enumerate(score['Class Accuracy: ']):
    print(f'Class[{i}] Accuracy:\t{_ca}')
    test_res_dict_ix[f'Class[{i}] Accuracy'] = _ca

    print(f'Class[{i}] IoU:     \t{class_iu[i]}')
    test_res_dict_ix[f'Class[{i}] IoU'] = class_iu[i]

Pixel Acc: 	0.9747437458948593
Mean Class Acc: 	0.8997274596024494
Freq Weighted IoU: 	0.9516404018943099
Mean IoU: 	0.8605054461994571
Class[0] Accuracy:	0.9942745116032634
Class[0] IoU:     	0.9827456173386825
Class[1] Accuracy:	0.9479334462858505
Class[1] IoU:     	0.9091408633323318
Class[2] Accuracy:	0.9900642968186641
Class[2] IoU:     	0.972814781910353
Class[3] Accuracy:	0.9447758919536424
Class[3] IoU:     	0.8975790572705358
Class[4] Accuracy:	0.9019024962335069
Class[4] IoU:     	0.7935355071700427
Class[5] Accuracy:	0.6194141147197688
Class[5] IoU:     	0.6072168501747967


In [ ]:
with open(pjoin('models', model_name, 'test_res_dict_ix.json'), 'w') as f:
    json.dump(test_res_dict_ix, f)